Start from scratch.... let's draw from one random distribution some number of times.

Then, add it to the result of a second distribution. Store the results

Then, consider time phase components

When these modules are developed, its ready to start plugging it in to excel

In [1]:
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt



import random

# Queuing-as df first


In [2]:
[int(sp.random.lognormal(mean=5, sigma=1))]

[56]

In [ ]:
call_time_dist

In [12]:
num_customers = 10
time_periods = 20
num_reps = 50

In [13]:
def generate_customer_table(seed=None):
    if seed is not None:
        np.random.seed(seed)

    df_customers = pd.DataFrame([i for i in range(1,num_customers+1)], columns=['customer_id'])
    df_customers['required_call_time'] = [np.random.choice([1,2,3,4,5]) for i in range(num_customers)]
    df_customers['time_in']=[np.random.choice(range(1,time_periods)) for i in range(num_customers)]
    df_customers['time_out']=[0 for i in range(num_customers)]
    df_customers['time_assigned'] = [0 for i in range(num_customers)]
    df_customers['assigned_rep'] = ['none' for i in range(num_customers)]

    df_customers=df_customers.sort_values(by='time_in')
    df_customers['customer_id'] = [i for i in range(1,num_customers+1)]
    
    return df_customers
# df_customers

def generate_qs():

    df_q = pd.DataFrame([0 for x in range(time_periods*2)], columns=['customer_q_list'])
    df_q['customer_q_list'] = [[] for x in range(time_periods*2)]
    df_q['update_event'] = [0 for x in range(time_periods*2)]

    return df_q

def generate_reps():
    df_reps = pd.DataFrame(columns=['rep1_busy', 'rep2_busy'])
    df_reps['rep1_busy']=[0 for x in range(time_periods*2)]
    df_reps['rep2_busy']=[0 for x in range(time_periods*2)]
    df_reps['rep3_busy']=[0 for x in range(time_periods*2)]
    #df_assigned = pd.DataFrame(columns=['rep1_busy_with', 'rep2_busy_with'], index=df_reps.index)
    
    return df_reps

In [14]:
def run_day(df_customers, df_q, df_reps, iteration):
    for t in range(time_periods*2):

        df_q.at[t, 'customer_q_list'] = list(df_customers[(df_customers.time_assigned==0)&(df_customers.time_in<=t)].customer_id)


        if len(df_q['customer_q_list'][t])!=0:

            for customer_qued in df_q['customer_q_list'][t]:

                for rep in df_reps.columns:
                    if df_reps[rep][t]==0:
                        _req_call_time = int(df_customers[df_customers.customer_id==customer_qued]['required_call_time'])
                        df_customers.at[df_customers.customer_id==customer_qued, ['time_assigned', 'time_out',  'assigned_rep']] = [t, t+_req_call_time , rep]

                        df_reps.loc[t: t + _req_call_time, rep] = 1
                        #df_assigned.loc[t: t + _req_call_time , rep+'_with'] = customer_qued
                        break
        df_customers['wait_time'] = df_customers.time_assigned - df_customers.time_in
        df_customers['iteration'] = iteration
        
        df_txn_log = pd.concat([df_q, df_reps],axis=1)
        df_txn_log['num_in_q'] = df_txn_log.customer_q_list.apply(lambda x: len(x))
        df_txn_log['iteration'] = iteration
        df_txn_log = df_txn_log.iloc[:df_customers.time_out.max()+1, :]
    return df_customers, df_txn_log

In [15]:
df_customers, df_txn_log = run_day(generate_customer_table(seed=42), 
                                   generate_qs(), 
                                   generate_reps(), 
                                   iteration=1)

In [16]:
for i in range(2,10):
    df_a, df_b = run_day(generate_customer_table(), 
                                   generate_qs(), 
                                   generate_reps(), 
                                   iteration=i)
    df_customers = pd.concat([df_customers, df_a])
    df_txn_log = pd.concat([df_txn_log, df_b])

In [17]:
# Wait Times
np.quantile(df_customers.wait_time, [.25, .5, .99])

array([0.  , 0.  , 3.11])

In [23]:
# Rep utilization rate
df_txn_log.loc[:mean()


update_event    0.000000
rep1_busy       0.751220
rep2_busy       0.570732
rep3_busy       0.448780
num_in_q        0.692683
iteration       5.112195
dtype: float64

In [19]:
df_txn_log

,customer_q_list,update_event,rep1_busy,rep2_busy,rep3_busy,num_in_q,iteration
0,[],0,0,0,0,0,1
1,[1],0,1,0,0,1,1
2,"[2, 3]",0,1,1,1,2,1
3,[4],0,1,1,1,1,1
4,"[4, 5]",0,1,1,1,2,1
5,"[4, 5]",0,1,1,1,2,1
6,"[5, 6]",0,1,1,1,2,1
7,[6],0,1,1,1,1,1
8,"[6, 7]",0,1,1,1,2,1
9,[7],0,1,1,1,1,1
